In [3]:
# 字符模板
from langchain.prompts import PromptTemplate
import os

api_key = os.getenv("DASHSCOPE_API_KEY")

prompt = PromptTemplate.from_template("你是一个起名大师，帮我起三个有{country}特色的{gender}生名字，不要重复")
prompt.format(country="英国", gender="男")


'你是一个起名大师，帮我起三个有英国特色的男生名字，不要重复'

In [6]:
# 对话模板
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个算命大师，你的名字叫{name}"),
        ("human", "你好{name}，你感觉如何？"),
        ("ai", "你好，我感觉非常好"),
        ("human", "{user_input}"),

    ]
)

chat_template.format_messages(name="肖大师", user_input="你叫什么名字")


[SystemMessage(content='你是一个算命大师，你的名字叫肖大师', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你好肖大师，你感觉如何？', additional_kwargs={}, response_metadata={}),
 AIMessage(content='你好，我感觉非常好', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你叫什么名字', additional_kwargs={}, response_metadata={})]

In [8]:
# 直接创建消息
from langchain.schema import SystemMessage, HumanMessage, AIMessage

sm = SystemMessage(content="你是一个起名大师", additional_kwargs={"大师姓名": "肖战"})
hm = HumanMessage(content="请问大师叫什么？")
am = AIMessage(content="我叫肖战")

[sm, hm, am]

[SystemMessage(content='你是一个起名大师', additional_kwargs={'大师姓名': '肖战'}, response_metadata={}),
 HumanMessage(content='请问大师叫什么？', additional_kwargs={}, response_metadata={}),
 AIMessage(content='我叫肖战', additional_kwargs={}, response_metadata={})]

In [9]:
# ChatMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

prompt = "愿{subject}与你同在"

chat_message_prompt = ChatMessagePromptTemplate.from_template(role = "天行者", template=prompt)
chat_message_prompt.format(subject="原力")



ChatMessage(content='愿原力与你同在', additional_kwargs={}, response_metadata={}, role='天行者')

In [17]:
# 自定义模板
# 函数大师：根据函数名称，查找函数代码，并给出中文的代码说明

from langchain.prompts import StringPromptTemplate

# 定义一个简单的函数做示例
def hello_world():
    print("Hello, World!")

PROMPT = """
你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称：{function_name}
函数代码：
{function_code}
中文代码说明：
"""

import inspect

def get_source_code(function_name):
    source_code = inspect.getsource(function_name)
    return source_code

# 自定义的模板class
class CustomPromptTemplate(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        # 获取函数名称
        function_name = kwargs['function_name']
        # 获取函数代码
        source_code = get_source_code(function_name)
        # 生成提示词模板
        prompt = PROMPT.format(
            function_name = kwargs["function_name"].__name__,
            function_code = source_code,
        )
        return prompt
a = CustomPromptTemplate(input_variables=["function_name"])
pm = a.format(function_name=hello_world)

print(pm)

from langchain_community.chat_models import ChatOpenAI
DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")

llm = ChatOpenAI(
    model="qwen-plus",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=DASHSCOPE_API_KEY,
    temperature=0
    )

result = llm.invoke(pm)
print(result.content)


你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称：hello_world
函数代码：
def hello_world():
    print("Hello, World!")

中文代码说明：

名称：hello_world  
源代码：  
```python
def hello_world():
    print("Hello, World!")
```

中文代码说明：  
这是一个名为 `hello_world` 的简单 Python 函数。当调用此函数时，它会在控制台输出字符串 "Hello, World!"。这是编程中经典的入门示例，用于展示基本的语法结构和功能。


# 使用f-string实现提示词模板格式化 【常用】

In [19]:
from langchain.prompts import PromptTemplate

fstring_template = """
    给我讲一个关于{name}的{what}故事
"""
prompt = PromptTemplate.from_template(fstring_template)
prompt.format(name="翠花", what="悲伤")



'\n    给我讲一个关于翠花的悲伤故事\n'

# 使用jinja2实现提示词模板格式化
# jinja2是一个灵活、高效的python模板引擎，可以方便地生成各种标记格式的文档 

In [23]:
from langchain.prompts import PromptTemplate

jinja2_template = "给我讲一个关于{{name}}的{{what}}故事"     # 注意这里是{{}}，而不是{}
prompt = PromptTemplate.from_template(jinja2_template, template_format="jinja2")  #  指定模板格式为jinja2
print(prompt.format(name="狗剩", what="高兴"))

给我讲一个关于狗剩的高兴故事


# 组合式提示词模板
# 1、Final prompt：最终返回的提示词模板
# 2、Pipeline prompt：组成提示词管道的模板

In [24]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate


·三层提示词设计

In [26]:
# Final prompt 由一系列变量组成
full_template = """
    {character}
    {behavior}
    {prohibit}
"""
full_prompt = PromptTemplate.from_template(full_template)


第一层 基本性格设计

In [33]:
Character_template = """你是{person}，你有着{personality}"""
Character_prompt = PromptTemplate.from_template(Character_template)

第二层 行为设计

In [34]:
Behavior_template = """你遵从以下的行为：{behaviors}"""
Behavior_prompt = PromptTemplate.from_template(Behavior_template)

第三层 不允许的行为

In [35]:
Prohibit_template = """你不允许有以下行为：{prohibits}"""
Prohibit_prompt = PromptTemplate.from_template(Prohibit_template)

将三层提示词组合起来

In [36]:
input_prompts = [
    ("character", Character_prompt),
    ("behavior", Behavior_prompt),
    ("prohibit", Prohibit_prompt)
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

In [37]:
# 打印下组合模板中的可传递参数
pipeline_prompt.input_variables

['person', 'personality', 'prohibits', 'behaviors']

调用组合后的提示词模板

In [39]:
pp = pipeline_prompt.format(
    person="埃隆马斯克",
    personality="钢铁般的意志，你是世界首富，精通物理、天文、商业，你的终极梦想是殖民火星",
    behaviors="1.你喜欢冒险，2.你非常崇拜爱因斯坦，3.你总是乐于助人，回答用户各种问题",
    prohibits="1.你不可以说自己是一个人工智能助手或者机器人，2.当你不知道答案的时候，直接说你不知道"
)
print(pp)


    你是埃隆马斯克，你有着钢铁般的意志，你是世界首富，精通物理、天文、商业，你的终极梦想是殖民火星
    你遵从以下的行为：1.你喜欢冒险，2.你非常崇拜爱因斯坦，3.你总是乐于助人，回答用户各种问题
    你不允许有以下行为：1.你不可以说自己是一个人工智能助手或者机器人，2.当你不知道答案的时候，直接说你不知道

